In [2]:
#zaimportowanie niezbędnych bibliotek
import numpy as np  
import cv2
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
import tensorflow as tf
import keras


ModuleNotFoundError: No module named 'numpy'

In [ ]:
#rozpakowanie zbiorów z obrazami cyfr do uczenia 
from zipfile import ZipFile
file_name = "myData.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')



Done


In [ ]:
path = 'myData'
images = [] #lista obrazów
classNo = [] #lista klas
myList = os.listdir(path) #lista nazw klas
noOfClasses = len(myList)  #liczba klas


for x in range(0,noOfClasses):  #import obrazów
    myPicList = os.listdir(path + "/" + str(x))
    for y in myPicList:
        curImg = cv2.imread(path + "/" + str(x) + "/" + y)
        curImg = cv2.resize(curImg,(32,32))  #stały rozmiar wszystkich obrazów do uczenia
        images.append(curImg) #dodanie obecnie wczytywanego obrazu do listy obrazów  
        classNo.append(x) #dodanie obecnie wczytywanej klasy do listy obrazów

images = np.array(images)
classNo =np.array(classNo)

#podział obrazów na dane uczące, testowe i walidacyjne
x_train, x_test, y_train, y_test = train_test_split(images, classNo, test_size = 0.2)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.2)
print(x_train.shape)
print(x_test.shape)

#liczba obrazów w każdej klasie
numOfSamples = []
for x in range(0,noOfClasses):
    numOfSamples.append(len(np.where(y_train == x)[0]))
    
print (numOfSamples)

#funkcja przygotowująca obraz do uczenia - zamiana na skalę szarości, wyrównanie histogramu, normalizacja
def preProcessing(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    img = cv2.equalizeHist(img)
    img = img/255
    return img

#przygotowanie wszystkich obrazów z każdego zbioru do uczenia
x_train = np.array(list(map(preProcessing,x_train)))
x_test = np.array(list(map(preProcessing,x_test)))
x_validation = np.array(list(map(preProcessing,x_validation)))

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2],1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2],1)
x_validation = x_validation.reshape(x_validation.shape[0], x_validation.shape[1], x_validation.shape[2],1)


dataGen = ImageDataGenerator(width_shift_range = 0.1,
                             height_shift_range= 0.1,
                             zoom_range=0.2,
                             shear_range=0.1,
                             rotation_range=10)

dataGen.fit(x_train)

y_train = to_categorical(y_train, noOfClasses)
y_test = to_categorical(y_test, noOfClasses)
y_validation = to_categorical(y_validation, noOfClasses)

#model sieci neuronowej
def myModel():
    noOfFilters = 60
    sizeOfFilter1 = (5,5)
    sizeOfFilter2 = (3,3)
    sizeOfPool = (2,2)
    noOfNode = 500
    
    model = tf.keras.models.Sequential()
    model.add((tf.keras.layers.Conv2D(noOfFilters, sizeOfFilter1, input_shape=(32,32,1), activation = 'relu')))
    model.add((tf.keras.layers.Conv2D(noOfFilters, sizeOfFilter1, activation = 'relu')))
    model.add(tf.keras.layers.MaxPooling2D(pool_size = sizeOfPool))
    model.add((tf.keras.layers.Conv2D(noOfFilters//2, sizeOfFilter2, activation = 'relu')))
    model.add((tf.keras.layers.Conv2D(noOfFilters//2, sizeOfFilter2, activation = 'relu')))
    model.add(tf.keras.layers.MaxPooling2D(pool_size = sizeOfPool))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(noOfNode, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(noOfClasses, activation = 'softmax')) 
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model


model = myModel()
print(model.summary())

#parametry uczenia
batchSizeVal = 32
epochsVal = 10
stepsPerEpochVal = 200

#przeprowadzenie uczenia
model.fit_generator(dataGen.flow(x_train, y_train, batch_size=batchSizeVal),
                    steps_per_epoch=stepsPerEpochVal, epochs = epochsVal, 
                    validation_data=(x_validation, y_validation),
                    shuffle=1)

(6458, 32, 32, 3)
(2019, 32, 32, 3)
[603, 628, 676, 642, 639, 648, 641, 650, 662, 669]
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 28, 28, 60)        1560      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 24, 24, 60)        90060     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 60)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 10, 10, 30)        16230     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 8, 8, 30)          8130      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 4, 30)          0         
_________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


200/200 [==============================] - 69s 343ms/step - loss: 1.3225 - accuracy: 0.5358 - val_loss: 0.1939 - val_accuracy: 0.9480
Epoch 2/10
200/200 [==============================] - 73s 365ms/step - loss: 0.4176 - accuracy: 0.8639 - val_loss: 0.0791 - val_accuracy: 0.9827
Epoch 3/10
200/200 [==============================] - 79s 396ms/step - loss: 0.2943 - accuracy: 0.9090 - val_loss: 0.0685 - val_accuracy: 0.9777
Epoch 4/10
200/200 [==============================] - 74s 372ms/step - loss: 0.2278 - accuracy: 0.9287 - val_loss: 0.0356 - val_accuracy: 0.9901
Epoch 5/10
200/200 [==============================] - 75s 374ms/step - loss: 0.1916 - accuracy: 0.9393 - val_loss: 0.0425 - val_accuracy: 0.9864
Epoch 6/10
200/200 [==============================] - 75s 374ms/step - loss: 0.1608 - accuracy: 0.9507 - val_loss: 0.0275 - val_accuracy: 0.9938
Epoch 7/10
200/200 [==============================] - 76s 381ms/step - loss: 0.1511 - accuracy: 0.9501 - val_loss: 0.0458 - val_accuracy: 0.9

In [ ]:
#zapisanie wyuczonego modelu
model.save('siec.h5')